In [1]:
def f(x: float) -> float:
    '''Результат - число, в котором будет максимум 5 старших значущих цифр исходного.'''
    return float(f"{x:.5g}")

In [2]:
for test in [1.32123, 0., 1, 10221.45, -12.345123]:
    print(f(test))

1.3212
0.0
1.0
10221.0
-12.345


In [3]:
import csv
from decimal import Decimal
from datetime import datetime, timedelta
from pprint import pprint


def td_format(td_object):
    seconds = int(td_object.total_seconds())
    periods = [
        ('year', 60 * 60 * 24 * 365),
        ('month', 60 * 60 * 24 * 30),
        ('day', 60 * 60 * 24),
        ('hour', 60 * 60),
        ('minute', 60),
        ('second', 1)
    ]

    strings = []
    for period_name, period_seconds in periods:
        if seconds > period_seconds:
            period_value, seconds = divmod(seconds, period_seconds)
            has_s = 's' if period_value > 1 else ''
            strings.append("%s %s%s" % (period_value, period_name, has_s))

    return ", ".join(strings)


def get_stat(start_time=1614556800, end_time=1614643200, accuracy=60):
    
    stat_intervals = {
        "<1": "{}<1",
        "1-2": "1<{}<=2",
        "2-5": "2<{}<=5",
        "5+": "{}>5",
        None: "{} is None"
    }
    
    with open('test_assignment_data.csv', newline='') as file:
        reader = csv.reader(file, delimiter=';')
        next(reader)
        data = [{
            "pair": row[1],  # pair
            "type": row[2],  # order type
            "price": Decimal(row[3]),  # price
            "open_ts": float(row[6]), # open timestamp
            "close_ts": float(row[7]) # close timestamp
        } for row in reader]


    data = sorted(filter(lambda k: k['close_ts'] > start_time and k['open_ts'] < end_time, data),
                  key=lambda k: k['pair'].lower())

    order_book = {}
    for order in data:
        if not order_book.get(order['pair']):
            order_book[order['pair']] = {order_type: [{'ts': i, 'price': None} for i in range(start_time,
                                                                                         end_time,
                                                                                         accuracy)]
                                         for order_type in ['buy', 'sell']}

        for x in (i for i in order_book[order['pair']][order['type']]
                  if order['open_ts'] <= i['ts'] <= order['close_ts'] and
                     (i['price'] is None or
                      i['price'] < order['price'] and order['type'] == 'buy' or
                      i['price'] > order['price'] and order['type'] == 'sell')):
            x['price'] = order['price']

    result = {key: {spread: {'duration': timedelta(0),
                             'pct': 0}
                    for spread in stat_intervals.keys()}
              for key in order_book.keys()}
    for pair, data in order_book.items():
        for i in range(len(data['buy'])):
            sell = data['sell'][i]['price']
            buy = data['buy'][i]['price']
            pct = ((sell - buy) / sell) * 100 if buy and sell else None
            
            for key, equation in stat_intervals.items():
                try:
                    res = eval(equation.format(pct))
                except:
                    continue
                if res:
                    result[pair][key]['duration'] += timedelta(seconds=accuracy)
                    break

    full_interval = end_time - start_time
    
    for pair, data in result.items():
        for interval in stat_intervals.keys():
            data[interval]['pct'] = round((data[interval]['duration'].total_seconds()/full_interval)*100, 2)
    return result


In [4]:
stat = get_stat(1614556800, 1614643200)

In [5]:
print("{:<10} {:<10} {:<20} {:<10}".format('Pair', 'Spread (%)', 'Duration', 'Duration (%)'))
for pair, data in stat.items():
    for spread, spread_data in data.items():
        print("{:<10} {:<10} {:<20} {:<10}".format(pair, str(spread), td_format(spread_data['duration']), spread_data['pct']))

Pair       Spread (%) Duration             Duration (%)
DOT/USDT   <1                              0.0       
DOT/USDT   1-2                             0.0       
DOT/USDT   2-5        23 hours, 58 minutes 99.86     
DOT/USDT   5+                              0.0       
DOT/USDT   None       2 minutes            0.14      
ETH/USD    <1                              0.0       
ETH/USD    1-2                             0.0       
ETH/USD    2-5        23 hours, 50 minutes 99.31     
ETH/USD    5+         4 minutes            0.28      
ETH/USD    None       6 minutes            0.42      
TON/USDT   <1                              0.0       
TON/USDT   1-2                             0.0       
TON/USDT   2-5                             0.0       
TON/USDT   5+         23 hours, 56 minutes 99.72     
TON/USDT   None       4 minutes            0.28      
XTZ/USD    <1                              0.0       
XTZ/USD    1-2                             0.0       
XTZ/USD    2-5        22 m